In [92]:
import numpy as np
import torch
import torchvision
from torchvision import transforms, datasets, models
print(torchvision.__version__)
from torch import nn as nn
from torch.utils.data import DataLoader, ConcatDataset
from sklearn.metrics import roc_auc_score, f1_score
import matplotlib.pyplot as plt
%matplotlib inline

0.13.0+cpu


In [93]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(DEVICE)

cpu


# 1 Reziser les images pour fitter dans le modèle

Tableau de référence pour les inputs resolution des différents EfficientNet
https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/

In [94]:
efficient_net_nb = 0
nb_to_res = {
    0: 224,
    1: 240,
    2: 260,
    3: 300,
    4: 380,
    5: 456,
    6: 528,
    7: 600,    
}
resolution = (nb_to_res[efficient_net_nb], nb_to_res[efficient_net_nb])

# 2 Normalisation des couleurs

Prendre celui qui performe le mieux

In [95]:
def to_uint8(x):
  return x.type(torch.uint8)

def to_float(x):
  return x.type(torch.FloatTensor)

normalisation = transforms.Compose([
      transforms.ToTensor(),                      
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # A modifier
      transforms.Resize(resolution)                                          
])

# 3 Data Augmentation

j'utilise RandAugment pour augmenter les données

Ekin D Cubuk, Barret Zoph, Jonathon Shlens, and Quoc V Le. Randaugment: Practical data
augmentation with no separate search. arXiv preprint arXiv:1909.13719, 2019.

In [96]:
rand_augment = transforms.Compose([
     transforms.RandAugment(),
      normalisation,
      transforms.Lambda(lambda x: to_uint8(x))      
])

# 5 Créer DataLoaders

In [135]:
def creer_datasets(dataset_path: str, train_size: float = 0.9, data_augmentation: bool = True, seed: int = 69):
    
    train_dataset = datasets.ImageFolder(root=rf"{dataset_path}\train", transform=normalisation)
    if data_augmentation:
        rand_augmented_dataset = datasets.ImageFolder(root=rf"{dataset_path}\train", transform=rand_augment)
        train_dataset = ConcatDataset([train_dataset, rand_augmented_dataset])
    len_train_set = int(train_size*len(train_dataset))
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [len_train_set, len(train_dataset)-len_train_set], generator=torch.Generator().manual_seed(seed))
    test_dataset = datasets.ImageFolder(root=rf"{dataset_path}\test", transform=normalisation)
    
    return train_dataset, val_dataset, test_dataset

def creer_dataloader(train_dataset: datasets.ImageFolder, val_dataset: datasets.ImageFolder, test_dataset: datasets.ImageFolder, batch_size: int = 32):
    
    pin_memory = bool(torch.cuda.is_available())
   
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=pin_memory)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)
    
    return train_loader, val_loader, test_loader

In [137]:
OUT_DIR = ".\Industrial Optical Inspection (reduced datasets)\Dataset texture"
train_dataset, val_dataset, test_dataset = creer_datasets(OUT_DIR, train_size=0.9, data_augmentation=True)
train_loader, val_loader, test_loader = creer_dataloader(train_dataset, val_dataset, test_dataset, batch_size=64)

# 6 Charger un modèle

In [130]:
def load_efficientnet(num_classes: int, efficient_net_nb: int = 0, pretrained: bool = True, freeze_batch_norm: bool = True):
    
    if efficient_net_nb == 0:
        model = models.efficientnet_b0(pretrained=pretrained)
    elif efficient_net_nb == 1:
        model = models.efficientnet_b1(pretrained=pretrained)
    elif efficient_net_nb == 2:
        model = models.efficientnet_b2(pretrained=pretrained)
    elif efficient_net_nb == 3:
        model = models.efficientnet_b3(pretrained=pretrained)
    elif efficient_net_nb == 4:
        model = models.efficientnet_b4(pretrained=pretrained)
    elif efficient_net_nb == 5:
        model = models.efficientnet_b5(pretrained=pretrained)
    elif efficient_net_nb == 6:
        model = models.efficientnet_b6(pretrained=pretrained)
    elif efficient_net_nb == 7:
        model = models.efficientnet_b7(pretrained=pretrained)
        
    if freeze_batch_norm:
        for module in model.modules():
            if isinstance(module, nn.BatchNorm2d):
                if hasattr(module, 'weight'):
                    module.weight.requires_grad_(False)
                if hasattr(module, 'bias'):
                    module.bias.requires_grad_(False)
    
    model.classifier = nn.Sequential(nn.Dropout(p=0.5, inplace=True),
                                    nn.Linear(in_features=model.classifier[1].in_features, out_features=num_classes, bias=True),
                                    nn.Softmax(dim=1))
    
    return model

model = load_efficientnet(len(test_dataset.classes), efficient_net_nb=efficient_net_nb, pretrained=True)

c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# 7 Entrainer le Modèle

In [154]:
def caculate_auc(predictions:torch.Tensor, labels:torch.Tensor) -> float:
    
    one_hot_labels = one_hot(labels, num_classes=predictions.shape[1])
    return roc_auc_score(one_hot_labels, predictions, multi_class="ovr", average="macro")

def one_hot(a: np.array, num_classes: int) -> np.array:
    
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)]).astype(int)
  
def train_model(model: nn.Module, train_loader: DataLoader, epochs: int = 10, lr: float = 0.001, weight_decay: float = 0.0001, device: torch.device = DEVICE):
    
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        batch_loss = []
        batch_outputs = []
        batch_labels = []
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.item())
            batch_outputs.append(outputs.detach().cpu().numpy())
            batch_labels.append(labels.detach().cpu().numpy())
        scheduler.step(metrics=loss.item())
        print(f"""Epoch: {epoch} ==> Loss moyenne: {sum(batch_loss)/len(batch_loss):.3f}
              | AUC: {caculate_auc(np.concatenate(batch_outputs, axis=0), np.concatenate(batch_labels, axis=0)):.3f}""")
    return model     
    
def eval_model(model: nn.Module, loader: DataLoader, device: torch.device = DEVICE):
    
    model.eval()
    predictions = []
    labels = []
    with torch.no_grad():
        for images, labels_ in loader:
            images = images.to(device)
            labels_ = labels_.to(device)
            outputs = model(images)
            predictions.append(outputs.detach().cpu().numpy())
            labels.append(labels_.detach().cpu().numpy())
    predictions = np.concatenate(predictions, axis=0)
    labels = np.concatenate(labels, axis=0)
    print(f"AUC: {caculate_auc(predictions, labels):.3f}")
    return predictions, labels


In [155]:
model = train_model(model, train_loader, epochs=2)
predictions, labels = eval_model(model, test_loader)

Epoch: 0 ==> Loss moyenne: 1.547 | AUC: 0.995
Epoch: 1 ==> Loss moyenne: 1.563 | AUC: 0.995


In [ ]:
SAVE_DIR = r"."
torch.save(model.state_dict(),SAVE_DIR)